In [1]:
import os
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point
import fiona

In [2]:
network_file = 'Chatt_Master.shp'
network_path = os.path.join('..', 'Data' )

In [3]:
network = gpd.read_file(os.path.join(network_path, network_file))
network = network[['ID', 'geometry']]

network['A'] = [pt['geometry'].coords[0] for i,pt in network.iterrows()]
network['B'] = [pt['geometry'].coords[-1] for i,pt in network.iterrows()]

# network = network.drop(columns='geometry')

# print(network.crs)
# print('Check for duplicates:', network['ID'].duplicated().any())
# network['ID'].describe()

In [ ]:
network_stream = []

for i, link in network.iterrows():
#     pt = Point(list(link['geometry'].coords)[0])
#     print(pt)
#     pt_buff = pt.buffer(0.0001)
#     print(pt_buff)
#     print(network[network['geometry'].intersects(pt)])
    net = network[network['geometry'].intersects(link['geometry'])]
    
    for j, stream in net.iterrows():
        if i!= j:
            if link['A']==stream['B']:
                print(link['ID'], 'downstream', stream['ID'])
                network_stream.append([link['ID'], 'downstream', stream['ID']])
            elif link['B']==stream['A']:
                print(link['ID'], 'upstream', stream['ID'])
                network_stream.append([link['ID'], 'upstream', stream['ID']])

network_stream = pd.DataFrame(network_stream, columns=['id', 'stream', 'stream_id'])

In [6]:
network_stream.to_csv(os.path.join(network_path, 'network_links.csv'), index=False)